In [1]:
import torch
import torch.nn as nn
from forensic_path10 import BYOL
from timm.models.layers import trunc_normal_
from einops.layers.torch import Rearrange
from swin_relative_trans import Block
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad,LayerCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import random
import numpy as np
import torchvision
torch.manual_seed(1412)
random.seed(1412)
np.random.seed(1412)
torch.cuda.manual_seed(1412)  
torch.cuda.manual_seed_all(1412) 

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
model = BYOL()
backbone = model.online_net.backbone
pool_layer = nn.AdaptiveAvgPool2d(1)
flatten_ = torch.nn.Flatten(1)
head = nn.Linear(backbone.out_channels, 7)
model = nn.Sequential(backbone, pool_layer, flatten_, head)
checkpoint = torch.load("./checkpoints/FPath-linate-mouse22-objcovstd99-100.pth", map_location='cpu')
model.load_state_dict(checkpoint['model'])
backbone = model[0].to(device)

In [4]:
# 相对位置编码+adaptive池化
class adaptivepool(torch.nn.Module):
    def __init__(self,  dim=128):
        super(adaptivepool, self).__init__()
        self.proj_att = nn.Linear(dim, dim, bias=False)
        self.proj_v = nn.Linear(dim, dim, bias=False)
        self.avgpool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):  # （b,n,c）
        x_ = x
        x = self.proj_att(x)
        x = x.transpose(1, 2)
        x = x.softmax(dim=-1)
        x_ = self.proj_v(x_)
        x_ = x_.transpose(1, 2)
        x = x * x_
        x = self.avgpool(x)

        return x
class Lineartransformer(nn.Module):
    """ transformer"""

    def __init__(self, dimm=2816, feat_dim=128, heads=16,num_classes=7):
        super(Lineartransformer, self).__init__()
        self.pro = nn.Linear(dimm, feat_dim)
        self.block = nn.Sequential(Block(dim=feat_dim,num_heads=heads))
        self.norm1 = nn.LayerNorm(feat_dim)
        self.norm2 = nn.LayerNorm(feat_dim)
        self.mlp_head = nn.Linear(feat_dim, num_classes)

        self.adaptivepool = nn.Sequential(adaptivepool())
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):  # (b,24,2816)
        x = self.pro(x)
        x = self.norm1(x)
        x = self.block(x)
        x = self.norm2(x)
        x = self.adaptivepool(x)
        x = torch.flatten(x, 1)
        x = self.mlp_head(x)

        return x

classifier = Lineartransformer()
classifier.load_state_dict(torch.load('./checkpoints/MIL-relative-adaptive(pre)-30.pth',map_location="cpu")["model"])
classifier = classifier.to(device)

In [5]:
###  relative_position adaptive
class MIL_relative(nn.Module):
    def __init__(self,):
        super(MIL_relative,self).__init__()
        self.model = backbone
        self.cls = classifier
        self.patch = nn.Sequential(Rearrange('b c (h p1) (w p2) -> (b h w) c p1 p2', p1=224, p2=224), )
        self.re_patch = nn.Sequential(nn.AdaptiveAvgPool2d(1),torch.nn.Flatten(1),Rearrange('(b h w) c -> b (h w) c', h=4, w=6))
    def forward(self,x):
        bs, c, h, w = x.size()
        x = self.patch(x)
        x = self.model(x)
        x = self.re_patch(x)
        x= self.cls(x)
        return x
    
nett=MIL_relative()

In [6]:
import cv2
import numpy as np
import torch
import timm
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    preprocess_image

In [7]:
def reshape_transform(tensor, height=4, width=6):
    result = tensor.reshape(tensor.size(0),
                            height, width, tensor.size(2))

    # Bring the channels to the first dimension,
    # like in CNNs.
    result = result.transpose(2, 3).transpose(1, 2)
    return result

def center_crop(img, dim):
	"""Returns center cropped image
	Args:
	img: image to be center cropped
	dim: dimensions (width, height) to be cropped
	"""
	width, height = img.shape[1], img.shape[0]

	# process crop width and height for max available dimension
	crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
	crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
	mid_x, mid_y = int(width/2), int(height/2)
	cw2, ch2 = int(crop_width/2), int(crop_height/2) 
	crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
	return crop_img

In [8]:
nett.eval()
nett = nett.cuda()
target_layers = [nett.cls.norm2]

In [9]:
nett.eval()
cam = LayerCAM(model=nett,target_layers=target_layers,
                                   use_cuda=True,
                                   reshape_transform=reshape_transform,
                                   )

In [10]:
import os
test_dataset = torchvision.datasets.ImageFolder(
    root=os.path.join('/home/omnisky/data1/shenxiaochen/data_all/train_data/'),
    transform=None)

In [11]:
for x, y in test_dataset.imgs:
    print(x)
    print(y)
    rgb_img = cv2.imread(x, 1)[:, :, ::-1]
    rgb_img = center_crop(rgb_img, (1344,896))
    rgb_img = np.float32(rgb_img) / 255
    input_tensor = preprocess_image(rgb_img, mean=[0.848, 0.524, 0.754],
                                    std=[0.117, 0.180, 0.127])
    grayscale_cam = cam(input_tensor=input_tensor,
                        targets=[ClassifierOutputTarget(y)],
                        eigen_smooth=True,
                        aug_smooth=True)
    grayscale_cam = grayscale_cam[0, :]
    cam_image = show_cam_on_image(rgb_img, grayscale_cam)
    cv2.imwrite('/home/omnisky/data1/shenxiaochen/layer_cam_miccai/{}_map.png'.format(x.split('/')[-1][:-4]), cam_image)

/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain10_1.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain10_2.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain10_3.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain10_4.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain10_5.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain13_0001.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain13_0002.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain13_0003.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain13_0004.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain13_0005.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain13_0006.png
0
/home/omnisky/data1/shenxiaochen/dat

/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain27_0006.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain28_0001.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain28_0002.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain28_0003.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain28_0004.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain28_0005.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain29_0001.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain29_0002.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain29_0003.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain29_0004.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain29_0005.png
0
/home/omnisky/data1/s

/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain47_0003.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain47_0004.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain47_0005.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain47_0006.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain48_0001.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain48_0002.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain48_0003.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain48_0004.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain48_0005.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain48_0006.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain49_0001.png
0
/home/omnisky/data1/s

/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0004.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0005.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0006.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0007.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0008.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0009.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0010.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0011.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain63_0012.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain64_0001.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain64_0002.png
0
/home/omnisky/data1/s

/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain79_0001.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain79_0002.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain79_0003.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain79_0004.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain79_0005.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain79_0006.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain79_0007.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain7_1.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain7_2.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain7_3.png
0
/home/omnisky/data1/shenxiaochen/data_all/train_data/brain _train/10X_brain7_4.png
0
/home/omnisky/data1/shenxiaochen/data

...

/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0003.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0004.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0005.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0006.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0007.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0008.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0009.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0010.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0011.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0012.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen77_0013.png
6
/home/omni

/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0002.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0003.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0004.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0005.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0006.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0007.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0008.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0009.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0010.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0011.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/40X_spleen85_0012.png
6
/home/omni

/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen22_0006.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen23_0001.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen23_0002.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen23_0003.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen23_0004.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen24_0001.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen24_0002.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen24_0003.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen24_0004.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen24_0005.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen26_0001.png
6
/home/omnisky/data1/s

/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen43_0004.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen43_0005.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen45_0001.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen45_0002.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen45_0003.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen45_0004.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen45_0005.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen45_0006.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen48_0001.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen48_0002.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen48_0003.png
6
/home/omnisky/data1/s

/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen75_0005.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen77_0001.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen77_0002.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen77_0003.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen77_0004.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen77_0005.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen77_0006.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen78_0001.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen78_0002.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen78_0003.png
6
/home/omnisky/data1/shenxiaochen/data_all/train_data/spleen_train/5X_spleen78_0004.png
6
/home/omnisky/data1/s